# Bot for trading ***USDT with crossing moving averages


### THIS BOT IS GOING TO OPERATE WITH CROSSING MOVING AVERAGE LINES WHEN:

#### PRICE > MA5 > MA10 > MA20

### Installs

In [1]:
#%pip install python-binance 
#%pip install numpy
#%pip install pandas
#%pip install IPython.display

### Imports

In [1]:
import config
import math
from binance.client import Client
from binance.enums import *
import time
import numpy as np
from IPython.display import clear_output

cliente = Client(config.API_KEY, config.API_SECRET, tld='com')


# Definition of coin and parameters

In [2]:
simbolo = 'BTCUSDT'   #Name of specific Symbol we are going to trade

simboloBalance = 'BTC'

quantityOrder = 15 # Quantity of USDT I am going to buy per operation. Around 15$...

decimales = '{:.2f}' # Number of decimals in buy parameters, if there is too many decimals depending on the cryptocurrency there will be an error of PRICEFILTER 

decimals = 5


# Functions

## ***** MOVING AVERAGES WITH 5 MINS CANDLESTICKS ******



### Moving Average 5 in periods of 5mins during 25 minutes

In [3]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma5_local = sum / 5

    return ma5_local

### Moving Average 10 in periods of 5mins during 50 minutes

In [4]:
def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma10_local = sum / 10

    return ma10_local

### Moving Average 20 in periods of 5mins during 100 minutes

In [5]:
def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma20_local = sum / 20

    return ma20_local

### Function to round down decimals


In [6]:
def round_decimals_down(number:float, decimals:int=4):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

### *** Loop operation for continuous trading ***

In [14]:
count = 0
while 1:
    
    for i in range(10):
        if count == 3:
            clear_output(wait=True)
            count = 0
    count += 1
    
    requestMinQtOrder = cliente.get_symbol_info(simbolo)
    ordenes = cliente.get_open_orders(symbol=simbolo)
    print("")
    print("-----------------------------------------------------")
    print("Actual open orders") 
    print(ordenes)              # If [] is returned ... there is no orders
    print("")
    
    # Checking available assets in account to place sell order when required
    quantityOrder = 0
    balance = cliente.get_asset_balance(asset=simboloBalance)
    quantityOrder += float(balance['locked']) + float(balance['free'])
    quantityOrder_left = round_decimals_down(quantityOrder, decimals)  
    #Quantity rounded down to specific decimals needed for specific crypto
    
   
    if(len(ordenes) != 0 ):   # If there is orders placed (OCO) print following info
        print('There is ' +str(len(ordenes)) + ' orders opened, no new buys')
        print("Quantity to sell   " + str(quantityOrder_left))
        print("Sell Price if it goes DOWN   " + ordenes[0]['price'])
        print("Sell Price if it goes UP   " + ordenes[1]['price'])
        print("-----------------------------------------------------")
        print("")
        time.sleep(10) # I send robot to rest for 10 seconds to check if market changed.
        continue

        
    # I obtain the price of the cryptocurrency I am going to trade and transform quantityOrder_USDT into quantityOrder
    list_of_tickers = cliente.get_all_tickers()
    for tick_2 in list_of_tickers:
        if tick_2['symbol'] == simbolo:
            symbolPrice = float(tick_2['price'])
            quantityOrder = round_decimals_down(quantityOrder_USDT/symbolPrice , decimals)
    # END of obtaining price and quantityOrder.
    
    # Give values in this specific moment to the mean averages for further check of conditions
    ma5  = _ma5_() 
    ma10 = _ma10_()
    ma20 = _ma20_()

    if(ma20 == 0 ): continue
    
    # Important to accomodate decimals, because if not it is going to give us different errors

    print("--------" + simbolo + "---------")
    print(" Actual price of " + simbolo + " is: " + str(decimales.format(symbolPrice)))  
    print("")
    print("*******************************")
    print(" Price to buy " + str(decimales.format(ma20*0.995)))
    print(" Price MA5 " + str(decimales.format(ma5)))
    print(" Price MA10 " + str(decimales.format(ma10)))
    print(" Price MA20 " + str(decimales.format(ma20)))
    print("*******************************\n")


    if (symbolPrice > ma5 and ma5 > ma10 and ma10 > ma20): # If conditions are met, orders are places in Binance
        
        print("Buying if there is no open orders")
        print("")

        order = cliente.order_market_buy(
            symbol = simbolo,
            quantity = quantityOrder
        )
        
        # Checking available assets in account to place sell order when required
        quantityOrder = 0
        balance = cliente.get_asset_balance(asset=simboloBalance)
        quantityOrder += float(balance['locked']) + float(balance['free'])
        quantityOrder_left = round_decimals_down(quantityOrder, decimals)  
        #Quantity rounded down to specific decimals needed for specific crypto

        time.sleep(5) #Robot to wait 5 secs so buy order is already in Binance exchange!!

        # Place OCO order
        print("")
        print("PLACING OCO ORDER")
        print("StopLimitPrice >   " +str(decimales.format(symbolPrice*0.985)),)
        print("Quantity >   " + str(quantityOrder_left))
        print("StopPrice >   " + str(decimales.format(symbolPrice*0.99)))
        print("Price >   " + str(decimales.format(symbolPrice*1.01)))
        print("")
        print("-----------------------------------------------------")
        print("")
        
        ordenOCO = cliente.create_oco_order(
            symbol = simbolo,
            side = SIDE_SELL,
            stopLimitPrice = str(decimales.format(symbolPrice*0.985)),  # Stop Loss
            stopLimitTimeInForce = TIME_IN_FORCE_GTC,
            quantity = quantityOrder_left, 
            stopPrice = str(decimales.format(symbolPrice*0.99)),
            price = str(decimales.format(symbolPrice*1.01)), # Take Profit
            )
    
        time.sleep(10) # Robot goes to rest after placing order to wait for market to change...

    else: 
        print("Conditions are not met")
        print("-----------------------------------------------------")
        print("")
        time.sleep(10) # Robot goes to rest after not meeting conditions to buy to wait for market to change...
    
    


-----------------------------------------------------
Actual open orders
[{'symbol': 'BTCUSDT', 'orderId': 16460932302, 'orderListId': 79513249, 'clientOrderId': '17DyIJ5KeHjdR4Lt1jhXW7', 'price': '17468.42000000', 'origQty': '0.00082000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'STOP_LOSS_LIMIT', 'side': 'SELL', 'stopPrice': '17557.10000000', 'icebergQty': '0.00000000', 'time': 1671086734173, 'updateTime': 1671086734173, 'isWorking': False, 'workingTime': -1, 'origQuoteOrderQty': '0.00000000', 'selfTradePreventionMode': 'NONE'}, {'symbol': 'BTCUSDT', 'orderId': 16460932303, 'orderListId': 79513249, 'clientOrderId': 'uomypGFa1JlsoDSrmKHAgu', 'price': '17911.78000000', 'origQty': '0.00082000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT_MAKER', 'side': 'SELL', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1671086734173, 'u

KeyboardInterrupt: 

# DEBUGGING


In [ ]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma5_local = sum / 5

    return ma5_local

def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma10_local = sum / 10

    return ma10_local

def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma20_local = sum / 20

    return ma20_local

ma5  = _ma5_()
ma10 = _ma10_()
ma20 = _ma20_()

print(ma5 , ma10, ma20)

In [ ]:
requestMinQtOrder = cliente.get_symbol_info(simbolo)

requestMinQtOrder

In [ ]:
requestMinQtOrder = cliente.get_symbol_info(simbolo)

requestMinQtOrder

int(math.floor(float(requestMinQtOrder['filters'][1]['minQty'])))

In [ ]:
balances = cliente.get_account()

balances

In [ ]:
quantityOrder

quantity = quantityOrder*0.999
quantity

In [9]:
status = cliente.get_account_api_trading_status()
    
status

{'data': {'isLocked': False,
  'plannedRecoverTime': 0,
  'triggerCondition': {'UFR': 300, 'IFER': 150, 'GCR': 150},
  'updateTime': 0}}

In [16]:
info = cliente.get_account()

#info

In [13]:
balance = cliente.get_asset_balance(asset='BTC')
balance


{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00082000'}